# AI for BI

Analyse data and provide a BI report including charts based on that data

We'll use CrewAI for the processing. This includes a Python interpreter which will be used to create the charts. You must have Docker installed locally to run the code automatically.

Running code generated by an LLM is potentially dangerous as there is n guarantee that the code wil be safe. Running the code in a Docker container, however, provides a sandbox for the code to run in and isolates it from the application code and local operating system.


Import the necessary libraries and set the LLM model (defaults to using OpenAI)

In [9]:
from crewai import Agent, Task, Crew

llm = "gpt-4o-mini"

In [10]:
files = [
    {
        'data_file_name':'sales_product_cat.csv',
        'chart_file_name': 'sales_product_summary.png',
        
    },
    {
        'data_file_name': 'monthly_sales.csv',
        'chart_file_name': 'monthly_sales.png',
    },
    {
        'data_file_name': 'sales_by_region.csv',
        'chart_file_name': 'sales_by_region.png',
    }
]

In [11]:
import plotly.express as px
import pandas as pd

df = pd.read_csv(files[0]['data_file_name'])
fig = px.bar(df,  x="Product Name", y="Revenue ($)", 
             color="Product Category", template="plotly_white")
fig.write_image(files[0]['chart_file_name'])

df = pd.read_csv(files[1]['data_file_name'])
fig = px.bar(df,  x="Month", y="Total Revenue ($)", 
             template="plotly_white")
fig.write_image(files[1]['chart_file_name'])

df = pd.read_csv(files[2]['data_file_name'])
fig = px.bar(df,  x="Region", y="Total Revenue ($)", 
             color = "Top-Selling Product", template="plotly_white")
fig.write_image(files[2]['chart_file_name'])

Import the tools required to read and write files

In [12]:
from crewai_tools import tool
from crewai_tools import FileReadTool, FileWriterTool

# Initialize the tool to read any files the agents knows or lean the path for
file_read_tool = FileReadTool()
file_writer_tool = FileWriterTool()

Set up the ``data_analysis_agent``

In [13]:
# Define the agent
data_analysis_agent = Agent(
        role="Data Analyser",
        goal="""You aim is to read and analyse sales data. You should
                then write a report on sales performance that includes an executive summary.
                """,
        backstory="You are assigned to perform sales analysis for a company",
        tools=[file_read_tool, file_writer_tool],
        allow_delegation=False,
        llm=llm,
        allow_code_execution=False
    )


In [14]:
query = f"""Read the Sales by product category data from the files:
             '{files[0]['data_file_name']}',
             '{files[1]['data_file_name']}',
             '{files[2]['data_file_name']}'.
             Charts that illustrate the data are in the following files:
             '{files[0]['chart_file_name']}',
             '{files[1]['chart_file_name']}',
             '{files[2]['chart_file_name']}'.    

             Write a sales report based on this data that includes the charts given 
             above and markdown tables.
             Charts created should be saved to local files and referred to in the report which should be in markdown format.
             
             Use the following as a template (the texts in square brackets are instructions):
             "      
                    ### 1. Executive Summary
                    
                    - Total sales revenue: $[total sales revenue here]
                    - Percentage increase/decrease from the previous month: [Percentage increase/decrease]
                    - Key highlights: 
                      - Best-selling product, 
                      - highest growth, 
                      - etc. 
                    
                    ### 2. Sales by Product Category
                    [include product category chart here]
                    [For all categories:]
                      Category: [Product Category Name]
                      - Total sales revenue: $[Total sales revenue for category]
                      - Percentage change from last month: [Percentage change from last month for category]
                      - Key products: [List of top-selling products in this category]

                    ### 3. Sales Trends Over Time
                    [include monthly sale chart here]
                    Monthly Sales Comparison:
                    - Analysis: Commentary on observed trends, e.g., "Sales show a strong upward trend due to the successful launch of [Product Name] in [Month]."
                    Quarterly Sales Growth:
                    - Analysis: [Commentary on the performance per quarter]
                    
                    ### 4. Regional Sales Performance
                    [include regional sales chart here]
                    [For each region]
                      Region: [Region Name]
                      - Total sales revenue: $[XXX,XXX]
                      - Top-performing products: [List products]

                    ### 5. Insights
                    [analyse the data to see if there are any insights that you can provide]
              "

              Save the complete report to a local file 'report.md'.
        """


In [15]:
write_report = Task(
    description= query,
    expected_output="""A markdown file""",
    agent=data_analysis_agent,
    tools=[file_read_tool, file_writer_tool]
)

# Define the crewwrite_overview
crew = Crew(
    agents=[data_analysis_agent],
    tasks=[write_report],
    verbose=True
)
result2 = crew.kickoff()
task_output = write_report.output

2024-11-12 18:18:24,990 - 21432 - __init__.py-__init__:538 - WARNING: Overriding of current TracerProvider is not allowed


# Agent: Data Analyser
## Task: Read the Sales by product category data from the files:
             'sales_product_cat.csv',
             'monthly_sales.csv',
             'sales_by_region.csv'.
             Charts that illustrate the data are in the following files:
             'sales_product_summary.png',
             'monthly_sales.png',
             'sales_by_region.png'.    

             Write a sales report based on this data that includes the charts given 
             above and markdown tables.
             Charts created should be saved to local files and referred to in the report which should be in markdown format.
             
             Use the following as a template (the texts in square brackets are instructions):
             "      
                    ### 1. Executive Summary
                    
                    - Total sales revenue: $[total sales revenue here]
                    - Percentage increase/decrease from the previous month: [Percentage increase/d

In [16]:
#print(f"Task Description: {result.description}")
#print(f"Task Summary: {result.summary}")
print(f"Raw Output: {result2.raw}")

Raw Output: ```markdown
### 1. Executive Summary

- Total sales revenue: $4,200,000
- Percentage increase/decrease from the previous month: +6.2%
- Key highlights: 
  - Best-selling product: Smartphone
  - Highest growth: Sofa (+12%)

### 2. Sales by Product Category
![Product Category Chart](sales_product_summary.png)
For all categories:
- **Category**: Electronics
  - Total sales revenue: $2,220,000
  - Percentage change from last month: +3%
  - Key products: Smartphone, Laptop, Smart TV
- **Category**: Home Appliances
  - Total sales revenue: $735,000
  - Percentage change from last month: +6.67%
  - Key products: Microwave, Refrigerator, Washing Machine
- **Category**: Furniture
  - Total sales revenue: $240,000
  - Percentage change from last month: +10%
  - Key products: Sofa, Bed Frame, Dining Table

### 3. Sales Trends Over Time
![Monthly Sales Chart](monthly_sales.png)
Monthly Sales Comparison:
- Analysis: Sales show a strong upward trend due to the successful launch of the Sm

# Sales Performance Overview

## Sales Data Summary

### Product Category Sales
| Product Category      | Product Name       | Units Sold | Revenue ($) | % Change (Month-over-Month) |
|----------------------|--------------------|------------|-------------|------------------------------|
| Electronics          | Smart TV           | 1500       | 450000      | +10%                         |
| Electronics          | Laptop             | 1200       | 840000      | +5%                          |
| Electronics          | Smartphone         | 2000       | 1200000     | -2%                          |
| Home Appliances      | Refrigerator       | 700        | 350000      | +8%                          |
| Home Appliances      | Washing Machine     | 500        | 250000      | -3%                          |
| Home Appliances      | Microwave          | 900        | 135000      | +15%                         |
| Furniture            | Sofa               | 300        | 90000       | +20%                         |
| Furniture            | Dining Table       | 150        | 75000       | +12%                         |
| Furniture            | Bed Frame          | 250        | 125000      | +7%                          |

### Monthly Sales Overview
| Month      | Total Units Sold | Total Revenue ($) | % Change (Month-over-Month) |
|------------|------------------|--------------------|------------------------------|
| January    | 10000            | 3500000            | -                            |
| February   | 10500            | 3675000            | +5%                          |
| March      | 11000            | 3850000            | +4.8%                        |
| April      | 10800            | 3780000            | -1.8%                        |
| May        | 11500            | 4025000            | +6.5%                        |
| June       | 11300            | 3955000            | -1.7%                        |
| July       | 12000            | 4200000            | +6.2%                        |
| August     | 11800            | 4130000            | -1.7%                        |

## Charts

![Product Summary](./sales_product_summary.png)

![Monthly Sales](./monthly_sales.png)
